# Data Cleaning
Data cleaning is a crucial step in the data analysis process. It involves identifying and correcting (or removing) errors and inconsistencies in the data to improve its quality. Clean data leads to more accurate analyses and better decision-making. In this notebook, we will perform various data cleaning tasks on our dataset to prepare it for further analysis and modeling.

Our notebook will be structured in a way where we work per column of our dataset. Some columns will not be affected, we have written about this in our accountability report. 

## 0. Loading in dataset

In [95]:
from pathlib import Path
import pandas as pd

file_path = Path("../Data/Raw/Uitgebreide_VKM_dataset.csv")
dataset = pd.read_csv(file_path, low_memory=False)

## 1. name
We will use NLP (Natural Language Processing) on this column. The column is of type string.

## 2. shortdescription
This whole column will be dropped due to it's similarity with the column "module_tags". We chose to drop this column instead of the module_tags column due to the fact that in places where shortdescription holds on data, module_tags does.

In [96]:
dataset.drop(columns=["shortdescription"], inplace=True)
dataset.head()

,id,name,description,content,studycredit,location,contact_id,level,learningoutcomes,Rood,Groen,Blauw,Geel,module_tags,interests_match_score,popularity_score,estimated_difficulty,available_spots,start_date
0,159,Kennismaking met Psychologie,In deze module leer je hoe je gedrag van jezel...,In deze module leer je hoe je gedrag van jezel...,15,Den Bosch,58,NLQF5,A. Je beantwoordt vragen in een meerkeuze kenn...,4.0,2.0,1.0,5.0,"['brein', 'gedragsbeinvloeding', 'ontwikkeling...",0.54,319,1,79,2025-12-24
1,160,Learning and working abroad,Studenten kiezen binnen de (stam) van de oplei...,Studenten kiezen binnen de (stam) van de oplei...,15,Den Bosch,58,NLQF5,De student toont professioneel gedrag conform ...,5.0,3.0,1.0,1.0,"['internationaal', 'persoonlijke', 'ontwikkeli...",0.92,172,5,56,2025-12-20
2,161,Proactieve zorgplanning,Het Jeroen Bosch ziekenhuis wil graag samen me...,Het Jeroen Bosch ziekenhuis wil graag samen me...,15,Den Bosch,59,NLQF5,De student past pro actieve zorgplanning toe b...,NaN,NaN,NaN,NaN,"['proactieve', 'zorgplanning', 'cocreatie', 'z...",0.78,217,5,55,2025-09-23
3,162,Rouw en verlies,In deze module wordt stil gestaan bij rouw en ...,In deze module wordt stil gestaan bij rouw en ...,30,Den Bosch,58,NLQF6,De student regisseert en voert (deels) zelfsta...,NaN,NaN,NaN,NaN,"['rouw', 'verlies', 'palliatieve', 'zorg', 're...",0.69,454,1,54,2025-10-25
4,163,Acuut complexe zorg,In deze module kunnen studenten zich verdiepen...,In deze module kunnen studenten zich verdiepen...,30,Den Bosch,58,NLQF6,De student regisseert en voert (deels) zelfsta...,NaN,NaN,NaN,NaN,"['acute', 'zorg', 'complexiteit', 'ziekenhuis'...",0.40,178,5,38,2025-11-19


## 3. content
This column is in all rows, except 13 of them, a copy of the description column's data. For the 13 exceptions we will add the data to the description column and then drop the content column after. 

In [ ]:
# Masking where content and description differ
mask_diff = dataset["content"] != dataset["description"]

# Appending content data to description if they differ from eachother. 
def _merge_desc_content(row):
    desc = row["description"]
    cont = row["content"]
    if pd.isna(cont):
        return desc  # Do nothing
    if desc == cont:
        return desc # Dont append if the columns match
    
    # Appending content to description
    return str(desc) + " " + str(cont)

# Merging at places where content and description differ
dataset.loc[mask_diff, "description"] = dataset.loc[mask_diff].apply(
    _merge_desc_content, axis=1
)

# Showing all rows that changed, only their new description and their id value
updated_rows = dataset.loc[mask_diff, ["description", "id"]]
display(updated_rows.values)

# Content data can be dropped after appended to description column
dataset.drop(columns=["content"], inplace=True)

# Single row check
dataset[dataset['id'] == 179].values

array([['Deze verdiepende module gaat in op uiteenlopende aspecten die je binnen de oncologische zorg. We zorgen voor veel inbreng vanuit de dagelijkse praktijk door inzet van gastdocenten en excursies. Na deze module heb je uitgebreide inhoudelijke kennis over oncologische ziektebeelden, kun je klinisch redeneren bij kanker. Daarnaast leer je over de psychosociale gevolgen van de ziekte en wat voor begeleiding dit vraagt van zorgvragers en hun naasten. Oncologie',
        178],
       ['Ontwikkel een diepgaand begrip van klinisch redeneren in onze HBO-Verpleegkunde module Pro-Active Nursing. Leer anticiperen op zorgbehoeften door doordachte analyse en gezamenlijke, eenduidige klinische besluitvorming. Kom tot een professionele beoordeling van hoe het gaat met de zorgvrager en maak de klinische toestand inzichtelijk. Pro-active nursing',
        179],
       ['Maak impact daar waar het meest nodig is, met een focus op vroegtijdige signalering en begeleiding van jongeren met actuele pro

array([[179, 'Pro-active nursing',
        'Ontwikkel een diepgaand begrip van klinisch redeneren in onze HBO-Verpleegkunde module Pro-Active Nursing. Leer anticiperen op zorgbehoeften door doordachte analyse en gezamenlijke, eenduidige klinische besluitvorming. Kom tot een professionele beoordeling van hoe het gaat met de zorgvrager en maak de klinische toestand inzichtelijk. Pro-active nursing',
        15, 'Breda', 63, 'NLQF5', 'Ntb', nan, nan, nan, nan, "['ntb']",
        0.7, 185, 2, 42, '2025-10-02']], dtype=object)

## 

We drop the colour columns (Green, Blue, Red, Yellow) as they are not relevant for our recommendation system mainly because it is not useful metadata for understanding user preferences or item characteristics and most values in these columns are Not A Number (NaN).